In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [70]:
import dill
from IPython.core import display as ICD  # to print multiple nice pandas tables
import json
import logging
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import seaborn as sns
import tqdm
from tqdm import tqdm_notebook, tnrange
sns.set()
import os
import pyperclip

In [9]:
import datasets
import classify
from simple_model import SimpleModel
import glob
from testing import test_simple_model
import experimentsbatch as exp
import utils

In [5]:
from collections import defaultdict
results = defaultdict(dict)
dump = defaultdict(dict)

In [11]:
res = defaultdict(dict)
for f in glob.glob(exp.result_multiw_file_pattern.format('*')):
    ws = int(f.split('_')[-1].split('.')[0])
    r = pickle.load(open(f, 'rb'))
    for x in r:
        for y in r[x]:
            res[x][y+(ws,)] = r[x][y]

In [12]:
df = pd.DataFrame(res)

In [13]:
df.style.background_gradient(cmap=utils.cm)

In [22]:
crossvalidates = df.groupby(level=[0,1,2,3,4,6]).mean()

In [24]:
broadcasted = crossvalidates.copy()

In [29]:
two = broadcasted.loc[(slice(None),slice(None),slice(None),slice(None),slice(None),2),:]


In [44]:
steps = []
for lev in range(1,6):
    selected = broadcasted.loc[(slice(None),slice(None),slice(None),slice(None),slice(None),lev),:]
    selected = selected.reset_index(level=5).drop(columns='level_5')
    steps.append(selected)

In [74]:
all_tab = ""
for i in [1,4]:
    diff = (steps[i] - steps[0]).round(2)
    diff = diff.reset_index(level=[0,1,2,3])
    tex = diff[utils.NOTREC].to_latex()
    tab = utils.tabular(tex, 
                        'Accuracy increase for %d steps compared to 1 step'%(i+1), 
                        'tab:multiw:notrec:%s'%(i+1))
    all_tab+='\n\n'+tab
    tex = diff[utils.TREC].to_latex()
    tab = utils.tabular(tex, 
                        'Accuracy increase for %d steps compared to 1 step on TREC dataset'%(i+1),
                        'tab:multiw:trec:%s'%(i+1))
    all_tab+='\n\n'+tab
    
    ICD.display(diff[utils.NOTREC+utils.TREC].style.applymap(utils.color_positives))
    
final = all_tab.replace('Dataset','').replace('TREC-','').replace('   ',' ')
print(final)
pyperclip.copy(final)

,CRDataset,MPQADataset,MRDataset,SUBJDataset,TRECDataset-ABBR,TRECDataset-DESC,TRECDataset-ENTY,TRECDataset-HUM,TRECDataset-LOC,TRECDataset-NUM
test,-0.01,0,0.01,0.01,0,-0.01,-0.01,0,-0.01,0.01
train,0.02,0,0.01,-0,0,0,0.01,0,0,0
valid,-0.02,0.01,-0.01,0,-0,0.02,-0,0.01,0,-0.01


,CRDataset,MPQADataset,MRDataset,SUBJDataset,TRECDataset-ABBR,TRECDataset-DESC,TRECDataset-ENTY,TRECDataset-HUM,TRECDataset-LOC,TRECDataset-NUM
test,-0.01,0.01,0.02,0.01,0,-0.01,-0,-0,0,0.01
train,0.03,0.01,0.01,0.01,0,0.01,0.01,0,0.01,0.01
valid,-0.02,0.01,0,-0.01,0,0,0.02,-0.01,-0,0.01





\begin{table}[H]
\begin{center}

\begin{tabular}{lrrrr}
\toprule
{} &  CR &  MPQA &  MR &  SUBJ \\
\midrule
test  &  -0.01 &   0.00 &   0.01 &   0.01 \\
train &   0.02 &   0.00 &   0.01 &    -0.00 \\
valid &  -0.02 &   0.01 &  -0.01 &   0.00 \\
\bottomrule
\end{tabular}

\caption[Accuracy increase for 2 steps compared to 1 step]{Accuracy increase for 2 steps compared to 1 step}
\label{tab:multiw:notrec:2}
\end{center}
\end{table}




\begin{table}[H]
\begin{center}

\begin{tabular}{lrrrrrr}
\toprule
{} &  ABBR &  DESC &  ENTY &  HUM &  LOC &  NUM \\
\midrule
test  &     0.0 &     -0.01 &     -0.01 &     0.00 &    -0.01 &     0.01 \\
train &     0.0 &      0.00 &      0.01 &     0.00 &     0.00 &     0.00 \\
valid &      -0.0 &      0.02 &     -0.00 &     0.01 &     0.00 &    -0.01 \\
\bottomrule
\end{tabular}

\caption[Accuracy increase for 2 steps compared to 1 step on TREC dataset]{Accuracy increase for 2 steps compared to 1 step on TREC dataset}
\label{tab:multiw:trec:2}
\end{cen

In [75]:
dumps = defaultdict(dict)
for f in glob.glob(exp.dump_multiw_file_pattern.format('*')):
    ws = int(f.split('_')[-1].split('.')[0])
    r = pickle.load(open(f, 'rb'))
    for x in r:
        for y in r[x]:
            dumps[x][y+(ws,)] = r[x][y]

In [87]:
lens = defaultdict(dict)
for dataset in dumps:
    for steps in [1,2,3,4,5]:
        agg = 0
        for cval in range(3):
            agg += len(dumps[dataset][('batch', 'None', 0.1, cval, steps)]['train'])
        lens[dataset][(steps)] = agg/3

In [120]:
steps = pd.DataFrame(lens).astype(int)
steps.index.name='$m$'
all_tab = ''
all_tab += utils.tabular(utils.multireplace(steps[utils.NOTREC].to_latex()),
                        "Number of needed training steps for different $w'$",
                        'tab:multyw:steps'
                        )
all_tab += '\n\n'+utils.tabular(utils.multireplace(steps[utils.TREC].to_latex()),
                               "Number of needed training steps for different $w'$ on TREC dataset",
                                'tab:multyw:steps:trec'
                               )
print(all_tab)
pyperclip.copy(all_tab)


\begin{table}[H]
\begin{center}

\begin{tabular}{lrrrr}
\toprule
{} & CR & MPQA & MR & SUBJ \\
\$m\$ &      &       &      &       \\
\midrule
1  &     33 &      52 &     57 &      37 \\
2  &     32 &      43 &     45 &      46 \\
3  &     32 &      56 &     48 &      36 \\
4  &     33 &      68 &     63 &      39 \\
5  &     35 &      53 &     48 &      41 \\
\bottomrule
\end{tabular}

\caption[Number of needed training steps for different $w'$]{Number of needed training steps for different $w'$}
\label{tab:multyw:steps}
\end{center}
\end{table}




\begin{table}[H]
\begin{center}

\begin{tabular}{lrrrrrr}
\toprule
{} & ABBR & DESC & ENTY & HUM & LOC & NUM \\
\$m\$ &          &          &          &         &         &         \\
\midrule
1  &        32 &        38 &        38 &        72 &        38 &        48 \\
2  &        32 &        39 &        40 &        45 &        38 &        36 \\
3  &        32 &        48 &        35 &        38 &        32 &        38 \\
4  &        32 